In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5

In [ ]:
import numpy as np
from ICARUS.Database import DB
import matplotlib.pyplot as plt
from ICARUS.Environment.definition import EARTH_ISA
from ICARUS.Flight_Dynamics.state import State
from ICARUS.Computation.Solvers.AVL.analyses.pertrubations import avl_dynamic_analysis_fd, process_avl_fd_res

from Vehicles.Planes.hermes_449_parameterized import hermes_449


In [ ]:
# origin at chargo_start
const_cargo_length = 0.565
const_cargo_width = 0.160
const_node_offset_max = 0.3 #ask if its okay to use nose_x_max as fixed for wingspan_max
const_mass_per_wing = 0.35
const_total_rows = 9 #this gives the payload
const_payload_ydist = 0.04
const_payload_xdist = 0.06
const_payload_offset = 0.025 #offset of front / back payload
mass_tail = 0.15
mass_wing = 0.7 
masses_point = 3.6740 #TODO CHECK ME simiakes mazes
AR = 9
WCL = 14.9213
m = mass_tail + mass_wing + masses_point
S =(144e0 / 1550e0) * pow(((m * 35.27) / WCL), 2e0 / 3)
S=0.5
wingspan = np.sqrt(AR*S)


bounds = {
    "motor_x_clearance" : np.array([0.050, 0.220]),
    "nose_x" :np.array([-0.18 - const_payload_xdist/2, -const_node_offset_max - const_payload_xdist/2]),#xdist is because of where origin is 
    #tail
    "tail_pos_x" : np.array([const_cargo_length +  0.200, const_cargo_length + 0.800]),
    "vtail_elev_volume" : np.array([0.5, 0.6]), #
    "vtail_AR" : np.array([1., 2.]),
    "vtail_angle" : np.array([40, 60]),   
    #main wing
    "main_wing_LE_x_perc" : np.array([0., 1.]),
    "chord_1" : np.array([0.25, 0.4]),
    "chord_3" :np.array([0.1, 0.2]),
    "length_1_perc" : np.array([0.1, 0.7]),
    "length_3_perc": np.array([0.1, 0.7]),  #constraint for these 2
    "incidence_angle" : np.array([-2.,2.]),
    "twist" : np.array([-2., 0.]),
    "dihedral" : np.array([0., 2.]),
    "offset_perc": np.array([0., 1.]),
    #control surfaces
    "flap_L_perc" : np.array([0.1, 0.6]), 
    "flap_area_perc": np.array([0.05, 0.2]),
    "ail_area_perc" :np.array([0.05, 0.15]),
}

from numpy import random

i = 0 
while True:
    # Randomly sample the parameters
    params = {}
    for key in bounds.keys():
        params[key] = random.uniform(bounds[key][0], bounds[key][1])

    try:
        # Create the plane
        plane = hermes_449(
            name = 'name',
            motor_x_clearance = params["motor_x_clearance"],
            nose_x = params["nose_x"],
            tail_pos_x = params["tail_pos_x"],
            vtail_elev_volume = params["vtail_elev_volume"],
            vtail_AR = params["vtail_AR"],
            vtail_angle = params["vtail_angle"],
            main_wing_LE_x_perc = params["main_wing_LE_x_perc"],
            chord_1 = params["chord_1"],
            chord_3 = params["chord_3"],
            length_1_perc = params["length_1_perc"],
            length_3_perc = params["length_3_perc"],
            incidence_angle = params["incidence_angle"],
            twist = params["twist"],
            dihedral = params["dihedral"],
            offset_perc = params["offset_perc"],
        )
        break
    except:
        i+= 1

# Optimization

## Define Parameters

In [ ]:
from ICARUS.Core.types import ComplexArray, FloatArray

UINF = 20
solver2D = "Xfoil"
angles = np.linspace(-10, 10, 11)

# plane = hermes_449("hermes")
state = State(
        name="Unstick",
        airplane=plane,
        environment=EARTH_ISA,
        u_freestream= UINF
)

epsilons = {
    "u": 0.01,
    "w": 0.01,
    "q": 0.1,
    "theta": 0.01,
    "v": 0.01,
    "p": 0.1,
    "r": 0.1,
    "phi": 0.01,
}

state.add_all_pertrubations("Central", epsilons)
avl_dynamic_analysis_fd(plane, state, solver2D)
df = process_avl_fd_res(plane, state)
state.set_pertrubation_results(df)
state.stability_fd()
target_cl_cd = state.trim["CL/CD"]
print(target_cl_cd)

initial_longitudal_modes: ComplexArray = state.state_space.longitudal.eigenvalues
initial_lateral_modes: ComplexArray = state.state_space.lateral.eigenvalues

desired_lateral_modes: ComplexArray = np.array([
    -30. + 0.j,
    -1.0 + 4.j,
    -1.0 - 4.j,
    -0.5 + 0.j
], dtype=complex)
desired_lateral_omegas: FloatArray = np.abs(desired_lateral_modes)
desired_lateral_zetas: FloatArray = - desired_lateral_modes.real / desired_lateral_omegas


desired_longitudal_modes: ComplexArray = np.array([
    -1.0 - 4.j,
    -1.0 + 4.j,
    -0.1 - 0.4j,
    -0.1 + 0.4j
], dtype=complex
)
desired_longitudal_omegas: FloatArray = np.abs(desired_longitudal_modes)
desired_longitudal_zetas: FloatArray = - desired_longitudal_modes.real / desired_longitudal_omegas

## Define the optimization problem

In [ ]:
design_variables = [
    # Point Masses
    "payload_position_x",
    # Elevator
    "elevator_position_x",
    "elevator_root_chord",
    "elevator_tip_chord",
    "elevator_pitch",
    # Rudder
    "rudder_position_x",
    "rudder_root_chord",
    "rudder_tip_chord",
]

bounds = {
    # Point Masses
    "payload_position_x": (0., 0.5),
    # Elevator
    "elevator_position_x": (0.4, 0.8),
    "elevator_root_chord": (0.05, 0.4),
    "elevator_tip_chord": (0.01, 0.1),
    "elevator_pitch": (-3, 3),    
    # Rudder
    "rudder_position_x": (0.4, 0.8),
    "rudder_root_chord": (0.05, 0.4),
    "rudder_tip_chord": (0.05, 0.2),
}

desing_constants = {
    "state": state,
}

## Define Objective Function

In [ ]:
from ICARUS.Flight_Dynamics.trim import TrimNotPossible, TrimOutsidePolars
from ICARUS.Vehicle.plane import Airplane

def obj_fun(
    plane: Airplane,
    state: State,
):

    ## ACC CODE
    # PLANE
    # VRISKEIS 
    # PISTA
    # ACC SCORE
    # PENALTIES

    state.update_plane(plane)
    state.add_all_pertrubations("Central")
    try:
        avl_dynamic_analysis_fd(plane, state, "Xfoil")
    except (TrimNotPossible, TrimOutsidePolars):
        return np.inf

    df = process_avl_fd_res(plane, state)
    state.set_pertrubation_results(df)
    state.stability_fd()

    # longitudal_eigs: ComplexArray = unstick.state_space.longitudal.eigenvalues
    # lateral_eigs: ComplexArray = unstick.state_space.lateral.eigenvalues

    # Get the eigenvalues and the damping ratios for the longitudinal modes
    longitudal_omegas = state.state_space.longitudal.omegas
    longitudal_zetas = state.state_space.longitudal.zetas
    lateral_omegas = state.state_space.lateral.omegas
    lateral_zetas = state.state_space.lateral.zetas
  
    # Compute the fitness function
    fitness = 0.
    for i in range(len(desired_longitudal_modes)):
        fitness += (longitudal_omegas[i] - desired_longitudal_omegas[i])**2
        fitness += (longitudal_zetas[i] - desired_longitudal_zetas[i])**2

    return fitness

def constraint_cl_over_cd(
    state: State
):
    return state.trim["CL/CD"] - target_cl_cd + 1

def constraint_cm_a_pos(
    state: State
):
    if "Cm0" not in state.trim.keys():
        return np.inf
    return state.trim["Cm0"]

def wing_cuble_loading(
    plane: Airplane,
    state: State
):
    # Calculate the wing cube loading
    WCL = plane.mass / plane.main_wing.S**2
    return WCL

def constraint_rudder_position(
    plane: Airplane,
):
    # Set the position of the elevator and rudder
    eleve_position_x = plane.__getattribute__("elevator_position_x")
    plane.__setattr__("rudder_position_x", eleve_position_x)

In [ ]:
# plane.__setattr__("elevator_pitch", 90.0)
# plane.visualize()

## Run Optimizer

In [ ]:
from ICARUS.Optimization.Callbacks.design_variables_vis import DesignVariableVisualizer
from ICARUS.Optimization.Callbacks.eigenvalue_opt_vis import EigenvalueOptimizationVisualizer
from ICARUS.Optimization.Callbacks.optimization_progress_vis import OptimizationProgress
from ICARUS.Optimization.Callbacks.plane_geometry_vis import PlaneGeometryVisualization
from ICARUS.Optimization.Callbacks.plane_polars_vis import AirplanePolarOptimizationVisualizert
from ICARUS.Optimization.Optimizers.Airplane.airplane_dynamics_optimizer import Airplane_Dynamics_Optimizer

# Design variable callback
design_variable_callback = DesignVariableVisualizer(
    plane = plane,
    design_variables= design_variables,
    bounds= bounds,
)

# Progress callback
progress_callback = OptimizationProgress()

# Plane geometry callback
plane_geometry_callback = PlaneGeometryVisualization(plane = plane)

# Eigenvalue callback
eigenvalue_callback = EigenvalueOptimizationVisualizer(
    goal_longitudal= desired_longitudal_modes,
    goal_lateral= desired_lateral_modes,
    initial_longitudal= initial_longitudal_modes,
    initial_lateral= initial_lateral_modes
)

polar_callback = AirplanePolarOptimizationVisualizert(
    initial_state=state,
)

optimizer = Airplane_Dynamics_Optimizer(
    design_variables= design_variables,
    design_constants= desing_constants,
    plane= plane,
    state = state,
    f= obj_fun,
    non_linear_constraints = [
        {
            "type": "ineq",
            "fun": constraint_cl_over_cd,
            "lb": -1.,
            "ub": 1.
        },
        {
            "type": "ineq",
            "fun": constraint_cm_a_pos,
            "lb": 0.,
            "ub": np.inf
        },
        {
            "type": "ineq",
            "fun": wing_cuble_loading,
            "lb": 0.,
            "ub": 10.
        },
    ],
    bounds= bounds,
    jac= None,
    # jac_fun= jac_fun,
    callback_list= [
        eigenvalue_callback,
        design_variable_callback,
        progress_callback,
        polar_callback,
        # plane_geometry_callback
    ]
)

In [ ]:
res = optimizer("Nelder-Mead", options={"maxiter": 1000})

# Get Results

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

optimizer.current_obj.visualize(fig, ax) # type: ignore
optimizer.current_obj.visualize(fig, ax) # type: ignore

In [ ]:
plane_1: Airplane = optimizer.initial_obj
plane_2: Airplane = optimizer.current_obj

unstick_1 = State(name="Unstick", airplane=plane_1, environment=EARTH_ISA, u_freestream=UINF)
unstick_1.add_all_pertrubations("Central")
avl_dynamic_analysis_fd(plane_1, unstick_1, solver2D='Xfoil')
df = process_avl_fd_res(plane_1, unstick_1)
unstick_1.set_pertrubation_results(df)
unstick_1.stability_fd()

In [ ]:
unstick_2 = State(name="Unstick", airplane=plane_2, environment=EARTH_ISA, u_freestream=UINF)
unstick_2.add_all_pertrubations("Central")
avl_dynamic_analysis_fd(plane_2, unstick_2, solver2D="Xfoil")
df = process_avl_fd_res(plane_2, unstick_2)
unstick_2.set_pertrubation_results(df)
unstick_2.stability_fd()

In [ ]:
initial = (unstick_1.state_space.longitudal.zetas, unstick_1.state_space.longitudal.omegas)
current = (unstick_2.state_space.longitudal.zetas, unstick_2.state_space.longitudal.omegas)

In [ ]:
plt.scatter(initial[0], initial[1], label="initial")
plt.scatter(current[0],current[1], label="current")
plt.scatter(desired_longitudal_zetas,desired_longitudal_omegas,label="desired")
plt.grid()
plt.legend()
plt.show()